In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

#Image Data Generator w/ no augmentation
#Scaling for pixels
piece_train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

piece_test_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_valid_datagen = ImageDataGenerator(
    rescale = 1./255)


#Flow data from directory

piece_train_iter = piece_train_datagen.flow_from_directory(
    directory = 'data/color_data/train',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

piece_test_iter = piece_test_datagen.flow_from_directory(
    directory = 'data/color_data/test',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    shuffle=False,
    seed=42
)

piece_valid_iter = piece_valid_datagen.flow_from_directory(
    directory = 'data/color_data/valid',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

Found 1178 images belonging to 3 classes.
Found 318 images belonging to 3 classes.
Found 316 images belonging to 3 classes.


In [3]:
#Define NN architecture

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

piece_model = Sequential()
piece_model.add(Conv2D(filters=16, kernel_size=5, padding='same', activation='relu', 
                        input_shape=(135, 135, 1)))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
piece_model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Dropout(0.3))
piece_model.add(GlobalAveragePooling2D())
piece_model.add(Dense(2048, activation='relu'))
piece_model.add(Dense(2048, activation='relu'))
piece_model.add(Dropout(0.4))
piece_model.add(Dense(3, activation='softmax'))


piece_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 135, 135, 16)      416       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 67, 67, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 67, 67, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 67, 67, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 33, 33, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        8256      
__________

In [4]:
# compile the model
rms_opt = keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
piece_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

STEP_SIZE_TRAIN = piece_train_iter.n/piece_train_iter.batch_size
STEP_SIZE_VALID = piece_valid_iter.n/piece_valid_iter.batch_size



In [5]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='color_model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
piece_hist = piece_model.fit_generator(generator=piece_train_iter, 
                          steps_per_epoch=STEP_SIZE_TRAIN, 
                          validation_data=piece_valid_iter, 
                          validation_steps=STEP_SIZE_VALID,
                          epochs=100, 
                          callbacks=[checkpointer], 
                          verbose=2)

Epoch 1/100
Epoch 00000: val_loss improved from inf to 1.29167, saving model to color_model.weights.best.hdf5
4s - loss: 0.5999 - acc: 0.8189 - val_loss: 1.2917 - val_acc: 0.1582
Epoch 2/100
Epoch 00001: val_loss did not improve
2s - loss: 0.1488 - acc: 0.9569 - val_loss: 2.7141 - val_acc: 0.1519
Epoch 3/100
Epoch 00002: val_loss did not improve
2s - loss: 0.1402 - acc: 0.9557 - val_loss: 3.5737 - val_acc: 0.1519
Epoch 4/100
Epoch 00003: val_loss did not improve
2s - loss: 0.1601 - acc: 0.9523 - val_loss: 5.4662 - val_acc: 0.1582
Epoch 5/100
Epoch 00004: val_loss did not improve
2s - loss: 0.1379 - acc: 0.9611 - val_loss: 3.7424 - val_acc: 0.1772
Epoch 6/100
Epoch 00005: val_loss did not improve
2s - loss: 0.1181 - acc: 0.9635 - val_loss: 3.3387 - val_acc: 0.1361
Epoch 7/100
Epoch 00006: val_loss did not improve
2s - loss: 0.1141 - acc: 0.9665 - val_loss: 1.3593 - val_acc: 0.4684
Epoch 8/100
Epoch 00007: val_loss did not improve
2s - loss: 0.1291 - acc: 0.9692 - val_loss: 2.1348 - val_

Epoch 00066: val_loss did not improve
2s - loss: 0.0409 - acc: 0.9932 - val_loss: 0.4603 - val_acc: 0.8734
Epoch 68/100
Epoch 00067: val_loss did not improve
2s - loss: 0.0095 - acc: 0.9966 - val_loss: 0.5092 - val_acc: 0.8449
Epoch 69/100
Epoch 00068: val_loss did not improve
2s - loss: 0.0254 - acc: 0.9949 - val_loss: 0.0510 - val_acc: 0.9968
Epoch 70/100
Epoch 00069: val_loss did not improve
2s - loss: 1.0509e-04 - acc: 1.0000 - val_loss: 1.2801 - val_acc: 0.9051
Epoch 71/100
Epoch 00070: val_loss did not improve
2s - loss: 0.0334 - acc: 0.9958 - val_loss: 2.7533 - val_acc: 0.8259
Epoch 72/100
Epoch 00071: val_loss did not improve
2s - loss: 0.0384 - acc: 0.9937 - val_loss: 1.3772 - val_acc: 0.9146
Epoch 73/100
Epoch 00072: val_loss did not improve
2s - loss: 0.0453 - acc: 0.9905 - val_loss: 4.7932 - val_acc: 0.6962
Epoch 74/100
Epoch 00073: val_loss did not improve
2s - loss: 0.0499 - acc: 0.9899 - val_loss: 1.5345 - val_acc: 0.9019
Epoch 75/100
Epoch 00074: val_loss did not improv

In [6]:
#Reset test iterator
STEP_SIZE_TEST = piece_test_iter.n/piece_test_iter.batch_size
piece_test_iter.reset()
# load the weights that yielded the best validation accuracy
piece_model.load_weights('color_model.weights.best.hdf5')
# evaluate and print test accuracy
score = piece_model.evaluate_generator(generator=piece_test_iter,steps=STEP_SIZE_TEST)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 1.0


In [7]:
piece_test_iter.reset()
piece_pred = piece_model.predict_generator(piece_test_iter,steps=STEP_SIZE_TEST,verbose=1)

10/9 [==============================] - 0s    


In [8]:
predicted_class_indices=np.argmax(piece_pred,axis=1)
print(predicted_class_indices)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [9]:
labels = (piece_test_iter.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
truth =  [labels[k] for k in piece_test_iter.classes]

In [10]:
[labels[k] for k in piece_test_iter.classes]


['black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',
 'black',


In [11]:
piece_test_iter.filenames

['black/1538778538.3017461.jpg',
 'black/1538779431.4099524.jpg',
 'black/1538776272.1332486.jpg',
 'black/1539023040.9695048.jpg',
 'black/1538776153.1161563.jpg',
 'black/1539025486.6533895.jpg',
 'black/1539017658.0751133_28.jpg',
 'black/1539025921.7744703.jpg',
 'black/1538779519.5865016.jpg',
 'black/1538779739.058273.jpg',
 'black/1539023363.830272.jpg',
 'black/1539023428.3806713.jpg',
 'black/1539017657.8251727_33.jpg',
 'black/1539023228.1371589.jpg',
 'black/1539028085.6649253.jpg',
 'black/1539026872.2657654.jpg',
 'black/1538779127.0457888.jpg',
 'black/1539017656.0015128_29.jpg',
 'black/1539017658.0438766_13.jpg',
 'black/1538778901.7553732.jpg',
 'black/1538778196.1506379.jpg',
 'black/1539024583.6953154.jpg',
 'black/1539022975.1625128.jpg',
 'black/1538778101.779042.jpg',
 'black/1538779439.9036374.jpg',
 'black/1539025391.7579608.jpg',
 'black/1539023017.1607347.jpg',
 'black/1538779919.6389072.jpg',
 'black/1539024430.756946.jpg',
 'black/1539026923.465744.jpg',
 'b

In [12]:
filenames=piece_test_iter.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Truth": truth,
                      "Predictions":predictions})

In [13]:
results

,Filename,Predictions,Truth
0,black/1538778538.3017461.jpg,black,black
1,black/1538779431.4099524.jpg,black,black
2,black/1538776272.1332486.jpg,black,black
3,black/1539023040.9695048.jpg,black,black
4,black/1538776153.1161563.jpg,black,black
5,black/1539025486.6533895.jpg,black,black
6,black/1539017658.0751133_28.jpg,black,black
7,black/1539025921.7744703.jpg,black,black
8,black/1538779519.5865016.jpg,black,black
9,black/1538779739.058273.jpg,black,black
